#  <center><i>Twitter Sentiment Analysis with Natural Language Processing</i></center>

# <i>Steps I will be taking:</i>

### 1. Record Twitter Data
### 2. Data Pre Processing
### 3. Split Data between Training and Testing sets.
### 4. Implement Logistic Regression Model for Positive Tweet or Negatvie Tweet reads.
### 5. Feed Test set of data to the Trained Logistic Regression Model.


# <i>Libraries needed for this project</i>

### 1. Kaggle
### 2. Numpy
### 3. Pandas
### 4. Regular Expressions
### 5. Stop Words (from NLTK)
### 6. PorterStemmer (from NLTK)
### 7. TfidVectorizer (from SKLEARN)
### 8. TrainTestSplit (from SKLEARN)
### 9. LogisticRegression (from SKLEARN)
### 10. Accuracy_score (from SKLEARN)


# <center><i>Importing Twitter Sentimnent Analysis Dataset</i></center>

### <center>This is the sentiment140 dataset. It contains 1,600,000 tweets extracted using the twitter api . The tweets have been annotated (0 = negative, 2 - neutral, 4 = positive) and they can be used to detect sentiment.</center>

In [4]:
# API to fetch the dataset from Kaggle

!kaggle datasets download -d kazanova/sentiment140



Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


In [5]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score



In [6]:
import nltk

nltk.download("stopwords")

# print the stopwords in english ( words that dont add context to the NLP)

print(stopwords.words("english"))

# remove the stop words from the large dataset to make the model more efficient




['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\simon\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# <i>Data Processing</i>

In [9]:
# naming columns

columns = ["target", "id","date", "flag", "user", "Tweet"]

#loading the data from the pandas dataframe

twtData = pd.read_csv("C:/Users/simon/Python/Python_Lessons/twitter_dataset.csv", names = columns, encoding = "ISO-8859-1")

# interleaved_df = pd.concat([twtData[twtData['target'] == 0].iloc[:min(len(twtData[twtData['target'] == 0]), len(twtData[twtData['target'] == 1]))], twtData[twtData['target'] == 1].iloc[:min(len(twtData[twtData['target'] == 0]), len(twtData[twtData['target'] == 1]))]]).sort_index(kind='merge')


# check the number of rows and colunmns

twtData.shape # will result in (1600000, 6)

(1600000, 6)

In [14]:
# positive_df = twtData[twtData['target'] == 1]
# negative_df = twtData[twtData['target'] == 0]

# # Determine the minimum length to ensure equal alternation
# min_length = min(len(positive_df), len(negative_df))

# # Slice both DataFrames to the minimum length
# positive_df = positive_df.iloc[:min_length]
# negative_df = negative_df.iloc[:min_length]

# # Interleave the rows
# interleaved_df = pd.concat([positive_df, negative_df]).sort_index(kind='merge')

# # Reset the index
# interleaved_df.reset_index(drop=True, inplace=True)

# # Now you can use interleaved_df or further slice it if needed
# newdata = interleaved_df.iloc[:10000]

# # Apply the stemming function to this sample
# # newdata['stemmed_content'] = newdata['Tweet'].apply(stemming)

# # Check the result
# newdata.head()
# newdata.shape


In [13]:
newdata.head() # this prints the first 5 rows of the dataset by default parameter

,target,id,date,flag,user,Tweet


In [ ]:
# Check /count for missing values in the dataset

twtData.isnull().sum() # this sums all the missing values together to count them in a single code line



target    0
id        0
date      0
flag      0
user      0
Tweet     0
dtype: int64

In [ ]:
# Check the distribution of target column. How many 0s and 4s are therein the entire dataste?

twtData["target"].value_counts() # distribution is equal in this data

target
0    800000
4    800000
Name: count, dtype: int64

# <center><u>Important Note</u></center>

## <center><i>If the Distribution of a dataste is NOT equal, the machine learning model will not work properly for the project intended. <br> <br> Therefore it is necessary to use downsampling in order to equalise the distributions of the different sets of data for Optimization.</i></center>

In [ ]:
# Change the names of the labels, conveting themfrom 0 to "Positive" and 4 to "Negative"

twtData.replace({"target":{4:"Negative"}}, inplace = True)
twtData.replace({"target":{0:"Positive"}}, inplace = True)
twtData.head()



,target,id,date,flag,user,Tweet
0,Positive,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,Positive,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,Positive,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,Positive,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,Positive,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
positive_df = twtData[twtData['target'] == "Positive"]
negative_df = twtData[twtData['target'] == "Negative"]

positive_df = positive_df.iloc[:10000]
negative_df = negative_df.iloc[:10000]

newdata = pd.concat([negative_df, positive_df]) # Concatenated the 2 categories into a new smaller dataframe called newdata so that it's quicker to process.

In [ ]:
print(positive_df.shape)
negative_df.shape

(10000, 6)


(10000, 6)

# Stemming

## Process of reducing a word to its root word

In [11]:
# Reducing each word i the entire databae to their root forms with PorterStemmer

stemmed = PorterStemmer()

def stemming(content):

    stemmed_content = re.sub("[^a-zA-Z]", " ", content) # this removes all the characters that are not alphabetical
    stemmed_content = stemmed_content.lower() # thsi will turn allt he words into lower case
    stemmed_content = stemmed_content.split()
    stemmed_content = [stemmed.stem(word) for word in stemmed_content if not word in stopwords.words("english")]
    stemmed_content = " ".join(stemmed_content)

    return stemmed_content

# twtData["stemmed_content"] = twtData["Tweet"].apply(stemming) --> # 50 minutes to complete this execution as it takes into consideration ALL of the dataframe.

# So for the sake of time, I will use the .loc accessor to select the frist 20,000 rows of the dataframe only.



newdata["target"].value_counts()
newdata["stemmed_content"] = newdata["Tweet"].apply(stemming) # This will save in the "stemmed_content" column for those rows

NameError: name 'newdata' is not defined

In [ ]:
print(newdata["stemmed_content"], "\n\n\n####### BREAK ######\n\n\n", newdata["target"])

800000                     love health uandpet u guy r best
800001        im meet one besti tonight cant wait girl talk
800002    darealsunisakim thank twitter add sunisa got m...
800003    sick realli cheap hurt much eat real food plu ...
800004                         lovesbrooklyn effect everyon
                                ...                        
9995                                                aww sad
9996                           stupid dvd stuf good bit jaw
9997      dandi sephi close friend famili afraid work co...
9998                        crap look last tweet earli like
9999                                     anoth rainboot day
Name: stemmed_content, Length: 20000, dtype: object 


####### BREAK ######


 800000    Negative
800001    Negative
800002    Negative
800003    Negative
800004    Negative
            ...   
9995      Positive
9996      Positive
9997      Positive
9998      Positive
9999      Positive
Name: target, Length: 20000, dtype: object


## Separating Data and Label

In [ ]:
x = newdata["stemmed_content"].values
y = newdata["target"].values
print("Stemmed Content:\n", x, "\n")
print("Targets:\n", y)

Stemmed Content:
 ['love health uandpet u guy r best'
 'im meet one besti tonight cant wait girl talk'
 'darealsunisakim thank twitter add sunisa got meet hin show dc area sweetheart'
 ... 'dandi sephi close friend famili afraid work colleagu'
 'crap look last tweet earli like' 'anoth rainboot day'] 

Targets:
 ['Negative' 'Negative' 'Negative' ... 'Positive' 'Positive' 'Positive']


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, train_size=0.8, stratify=y, random_state=2)


print(x.shape, x_train.shape, x_test.shape)

print(x_train)
print("\n")
print(x_test)

(20000,) (16000,) (4000,)
['simultech yeah weird line long assum world continu fine without'
 'aponderingheart ye watch first episod far love love love love nd episod tonight'
 'sara xox leav make sad' ...
 'listen music talk besti amp make plan see hm movi friday'
 'seekin new job' 'hero soooo bore ugh found episod season']


['twitter wound'
 'nsingman reaction urban crime view held outsid citi con equival death penalti'
 'crush wyatt cenac much daili show suppos' ... 'sleep star'
 'sleepi star war loner' 'maxim megeld useless sleep fair want fun well']


In [ ]:
# converting the textual data to numerical data

vectorizer = TfidfVectorizer() # learns how many times each word is repeated and sets a value for importance based on that.
# This creates a bias from pattern recognition based on a word that is most present in a positive or negative sentence.

x_train = vectorizer.fit_transform(x_train) # transforms all the values of importance into the training data so that it is all the same.
x_test = vectorizer.transform(x_test) # only use fit_transform for training data.


In [ ]:
print(x_train)


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 119118 stored elements and shape (16000, 20096)>
  Coords	Values
  (0, 15814)	0.4381072802450677
  (0, 19827)	0.2356575557396753
  (0, 19133)	0.3138237953661589
  (0, 10115)	0.3239360884271173
  (0, 10277)	0.2376568868524926
  (0, 1050)	0.3772952363411329
  (0, 19510)	0.2615287190021183
  (0, 3517)	0.3417224710651225
  (0, 5990)	0.2969060869969978
  (0, 19385)	0.27739922435336506
  (1, 850)	0.36176943786571214
  (1, 19821)	0.19739043202681927
  (1, 19044)	0.16975754260193868
  (1, 6014)	0.1989742075409638
  (1, 5446)	0.4918071986566638
  (1, 5821)	0.228602451302166
  (1, 10359)	0.6044384571894548
  (1, 12067)	0.25236030185488223
  (1, 17853)	0.19643462210124493
  (2, 15102)	0.587854324427436
  (2, 19694)	0.5497621483278535
  (2, 9917)	0.390550622656313
  (2, 10613)	0.2987347707608602
  (2, 14992)	0.3322899658417758
  (3, 14992)	0.1828093476305661
  :	:
  (15996, 16766)	0.273763136980639
  (15996, 16465)	0.3929119386042842
  

In [ ]:
print(x_test)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 26103 stored elements and shape (4000, 20096)>
  Coords	Values
  (0, 18276)	0.42182268311695575
  (0, 19542)	0.9066783464978152
  (1, 3149)	0.2749160841012165
  (1, 3434)	0.3740247821248823
  (1, 4204)	0.2945152140509762
  (1, 7467)	0.3409505592120597
  (1, 12859)	0.24684517244101775
  (1, 13188)	0.38985208913922165
  (1, 14255)	0.3740247821248823
  (1, 18576)	0.38985208913922165
  (1, 18811)	0.2770265949524451
  (2, 3784)	0.5447862708548599
  (2, 3943)	0.5243856771028235
  (2, 11786)	0.31462080672047793
  (2, 15701)	0.3666629299976556
  (2, 16917)	0.44136110440495174
  (3, 8313)	0.6513933394426372
  (3, 12681)	0.47473074466740617
  (3, 16092)	0.43828722873878884
  (3, 16238)	0.39777096741940815
  (4, 2843)	0.5104509811556713
  (4, 4138)	0.23602257895681145
  (4, 6203)	0.4191387435919197
  (4, 9815)	0.3068519368628186
  (4, 13329)	0.37097495323324
  :	:
  (3995, 7766)	0.2255061398918757
  (3995, 7859)	0.20593030015122607
  (

## <i>Training the Machine Learning Model

In [ ]:
model = LogisticRegression(max_iter = 1000) # maximum number of times the model can go through the data to achieve the maximum accuracy

model.fit(x_train, y_train)

LogisticRegression(max_iter=1000)

## <i> Model Evaluation & Accuracy Score

In [ ]:
prediction = model.predict(x_train)
accuracy = accuracy_score(y_train, prediction)
 
print("Accuracy score of the TRAINING data:\n", accuracy) # Over 80% accuracy, pretty good

Accuracy score of the TRAINING data:
 0.849125


In [ ]:
prediction_test = model.predict(x_test)
accuracy_test = accuracy_score(y_test, prediction_test)
 
print("Accuracy score of the TESTING data:\n", accuracy_test) # Over 70% accuracy, still pretty good

Accuracy score of the TESTING data:
 0.7335


In [ ]:
import sklearn
from sklearn.metrics import f1_score
sklearn.metrics.f1_score(y_test, prediction_test , labels=None, pos_label=1, average='binary', sample_weight=None, zero_division='warn')

NameError: name 'y_test' is not defined

# <i> Saving model

In [ ]:
import joblib
#this allows saving and loading models

joblib.dump(model, "Twitter_model.joblib")
# file stored in the folder

['Twitter_model.joblib']

In [ ]:
x_new = x_test[0:20]
print(y_test[0:20])

print("\n\n")

prediction1 = model.predict(x_new)
print(prediction1)

print("\n\n")

accuracy1 = accuracy_score(y_test[0:20], prediction1)
print(accuracy1)  # 75% of accuracy, pretty good.

['Positive' 'Negative' 'Negative' 'Positive' 'Positive' 'Negative'
 'Positive' 'Negative' 'Positive' 'Negative' 'Negative' 'Negative'
 'Positive' 'Positive' 'Negative' 'Positive' 'Negative' 'Positive'
 'Negative' 'Negative']



['Negative' 'Positive' 'Positive' 'Positive' 'Positive' 'Negative'
 'Positive' 'Negative' 'Negative' 'Negative' 'Positive' 'Negative'
 'Positive' 'Positive' 'Negative' 'Positive' 'Negative' 'Positive'
 'Negative' 'Negative']



0.75


## <i> <center> The Natural Language Processing model has 75% accuracy in real time therefore it is pretty good at predicting results. And can be loaded through the joblib.import function. <br><br> <u> End of NLP Project</u>.